In [4]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('ProstateCancer.csv')

#Drop the unrelated columns
df = df.drop(columns=['Unnamed: 0.1','Unnamed: 0'])

#Seperate the training and testing data
train_mask = df.pop('train')

#center responses
y = df.pop('lpsa')
y = y-y.mean()
#standardize data
df = (df- df.mean())/df.std()
x_train = df[train_mask == 'T']
y_train = y[train_mask == 'T']
x_train = x_train.values
y_train = y_train.values

In [4]:

n,p = x_train.shape

In [11]:
mu = 0
#set all coefs to zero
theta = np.zeros(p)
for _ in range(p):
    #get the highest correlation
    corr = x_train.T@ (y_train - mu)
    corr_abs = abs(corr)
    corr_max = max(corr_abs)
    #get the active predictors
    active = np.isclose(corr_abs, corr_max)
    print(active)
    signs = np.where(corr[active]>0, 1, -1)
    

      
    
    
    x_active = signs * x_train[:,active]
    
    product = x_active.T @ x_active
    x_inv = np.linalg.inv(product)
    
    
    A = x_inv.sum() ** (-0.5)

    w = A * x_inv.sum(axis = 1)
    
    u = x_active @ w
    
    z = corr_max / A
    
    if not active.all():
        a = x_train.T@u
        
        invert = np.invert(active)
        
        cor_inverted = corr[invert]
        
        a_inverted = a[invert]
        
        next_predictors = np.concatenate([(corr_max - cor_inverted) / (A - a_inverted),
                                     (corr_max + cor_inverted) / (A + a_inverted)])
        z = next_predictors[next_predictors >= 0].min()
    
    
    mu += z * u
    
    
    theta[active] += z * signs

[ True False False False False False False False]
[ True  True False False False False False False]
[ True  True False False  True False False False]
[ True  True False False  True False False  True]
[ True  True False  True  True False False  True]
[ True  True  True  True  True False False  True]
[ True  True  True  True  True  True False  True]
[ True  True  True  True  True  True  True  True]


In [12]:
theta

array([10.08773104,  6.90604419, -1.97106193,  3.78566139,  5.28192838,
       -1.63588749, -0.17466915,  3.98160547])

In [13]:
y_test = y[train_mask=='F']
x_test = df[train_mask=='F']

In [19]:
np.mean((y_train-x_train@theta)**2)

338.26428015720535

In [1]:
from sklearn import linear_model

In [9]:
reg = linear_model.Lars(n_nonzero_coefs=8,normalize=False)

In [10]:
reg.fit(x_train,y_train)

Lars(copy_X=True, eps=2.220446049250313e-16, fit_intercept=True, fit_path=True,
     n_nonzero_coefs=8, normalize=False, precompute='auto', verbose=False)

In [14]:
y_test = y[train_mask=='F']
x_test = df[train_mask=='F']

In [16]:
np.mean((y_test-reg.predict(x_test))**2)

0.5212740055076